# Word-level classification with a Gradient-Boosting model

NOTE: the script can be copied to run in parallel on multiple cores.

In [1]:
import random
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import seaborn as sns

pd.options.display.float_format = '{:20,.5f}'.format
sns.set()

SEED = random.seed(0)
EXPERIMENTS = Path('experiments')
SUBSET = 'Train' # 'Test'
BASE_MODEL_NAME = 'GBC_POS-PMFT' # GradBoostClassifier model trained on dataset 'raw', with POS tag and 
                                    # PubMed data-trained FastText as features.    

# add required libraries etc
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
# train_val = pd.read_parquet('data/split/train.parquet')
train_val = pd.read_parquet('data/features/km_4-8-12_10x300.parquet')

# the test set is currently withheld (493 documents). Test using a validation split (see below).

In [3]:
%%notify -m "Finished one-hot coding columns"

def hotcode(df):
    

    num_cols = df._get_numeric_data().columns
    cat_cols = (set(df.columns) - set(num_cols)) - {'Word'}
    
    print(cat_cols)
    dummies = pd.get_dummies(df[cat_cols])
    
    print('hotcode complete.')
    # assert check that type is numeric for all
    
    return pd.concat([dummies, df[num_cols]], axis=1)

print('hotcoding categorical columns...')
try:
    train_val = hotcode(train_val)
except ValueError:
    print('No categorical values found in data')

hotcoding categorical columns...
set()
No categorical values found in data


<IPython.core.display.Javascript object>

# Parameters & Partitioning

The training and validation split is only an example for K-fold crossvalidation to simulate an 80:20 train/validation dataset. See code examples of  https://scikit-learn.org/stable/modules/cross_validation.html and https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html for more information.

In [4]:
k_folds = 5 # not implemented

pio = {"participants", "interventions", "outcomes"}
features = set(train_val.columns).difference(pio.union({'Word'}))

metrics = ['accuracy', 'precision', 'recall', 'f_score', 'support']

Validation split (KCV currently not implemented, so just a simulation)

In [5]:
train_idx, val_idx = train_test_split(train_val.index.unique('doc'), 
                                      train_size=1-(1/k_folds)) # default 70:30

print(f'splitting data {len(train_idx)}:{len(val_idx)}')

train_set = train_val.loc[(train_idx, slice(None)),:]
val_set   = train_val.loc[(val_idx, slice(None)),:]

X_train, X_test = train_set[features], val_set[features]
y_train, y_test = train_set[pio.union({'Word'})], val_set[pio.union({'Word'})]

splitting data 3600:900


In [6]:
# convert labels to string (pd.Categorical?)
y_train, y_test = [pd.concat([y[key].astype('str') for key in y],axis=1) for y in [y_train, y_test]]

print(f"Evluating model on validation set(s) of size {len(val_set)} with {len(val_set.index.unique('doc'))} documents")
print(f'Prepared dataset has {len(features)} features.')

Evluating model on validation set(s) of size 243684 with 900 documents
Prepared dataset has 120 features.


# Model

Train a model for each feature. Can also try a multiclass model, since only 3% of the labels overlap.

The model trains n_estimators, which are binary decision trees on a single feature. 'Boosting' refers to the fact that the trees learn to choose the most informative variables over generations, meaning the score gradually improves. The aggregate label prediction for each word is taken over all estimators. There are many tunable parameters for this model but for initial testing I've used 300 estimators and 416 features (meaning not all features will be explored, see the feature importance results).

The class weights are set with parameter `weight_factor`. If ` weight_factor=5`, then the class weight for the positive label is five times that of the negative label.

Highly advisable to do a test run with >3 estimators to see if all the results are saved correctly before running with a long training time.

In [7]:
BASE_MODEL_NAME = 'GBM'
time = datetime.now().strftime("%m%d-%H%M")
dir_name = BASE_MODEL_NAME  + f'_{time}'

# manually override the directory name. Careful, as you won't be warned if you are overwriting existing data.
# this is useful for running several notebooks in parallel (multi-processor)

dir_name = 'kM_clusttree_4-8-12'

print(f'Folder name set to {dir_name}')

Folder name set to kM_clusttree_4-8-12


In [8]:
targets = ['participants','interventions','outcomes']
n_estimators = 300
n_runs = 10
weight_factor = 5 # weight factor of positive class compared to negative

words = pd.read_pickle('data\\raw\\labels.pkl')

assert "Word" not in features
assert np.array([(label not in features) for label in pio]).all()

# use these overrides for multiple scripts
# targets = ['participants']
# targets = ['interventions']
# targets = ['outcomes']

In [ ]:
for run in range(n_runs):
    
    # cross-validation parameters defined here for now. Use sklearn.GridSearchCV for param tuning

    train_weights = y_train[pio].astype(int) * (1 - 1/weight_factor) + 1 / weight_factor
    test_weights = y_test[pio].astype(int) * (1 - 1/weight_factor) + 1 / weight_factor
        
    for target in targets:
        
        %notify -m f"Starting new iteration for {target}"    
            
        print(f"Training GradientBoosting model to predict {target} "
              f"on {len(X_train.index.unique('doc'))} training instances " 
              f"for label {target}")

        clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=1.0, 
                                         verbose=1, max_depth=1, random_state=0)
              
        clf = clf.fit(X_train, y_train[target], sample_weight=train_weights[target].values)
              
        score = clf.score(X_test, y_test[target], sample_weight=test_weights[target].values)
              
        y_pred = clf.predict(X_test)
        y_true = y_test[target].values 
              
        print(f'Saving data for run {run}, target {target} in {dir_name}')
              
        model_name = target[0].upper() + f'_wf{weight_factor}_run{run}'
        save_model = True
        
        if save_model:
            Path(f'models/{dir_name}').mkdir(exist_ok=True, parents=True)
            dump(clf, f'models/{dir_name}/{model_name}.joblib')
        
        exp_folder = EXPERIMENTS / dir_name / model_name
        exp_folder.mkdir(exist_ok=True, parents=True)

        feat_imp = pd.DataFrame(clf.feature_importances_, index=X_test.columns)
        train_loss = pd.DataFrame(clf.train_score_)
        feat_imp.sort_index().to_csv(exp_folder /  'feature_importance.csv')
        train_loss.to_csv(exp_folder / 'training_loss.csv')
        
        pred_results = pd.DataFrame([y_test['Word'].values, y_true, y_pred], 
                                    index=['Word', 'T', 'P']).T
        pred_results.to_csv(exp_folder / 'predictions.csv')
              
        results = precision_recall_fscore_support(y_true, y_pred, sample_weight=test_weights[target])
        results = pd.DataFrame.from_dict(results)
        results.index = metrics[1:]
        results.to_csv(exp_folder / 'results.csv')
        
        with (exp_folder / 'accuracy.txt').open('a') as f:
            f.write(f'{accuracy_score(y_true, y_pred, sample_weight=test_weights[target])}')
            f.write('\n')
            f.write(f'{features}')
            f.close()

print('Done')

<IPython.core.display.Javascript object>

Training GradientBoosting model to predict participants on 3600 training instances for label participants
      Iter       Train Loss   Remaining Time 
         1           1.3088            8.25m
         2           1.2746            8.28m
         3           1.2456            8.10m
         4           1.2259            7.96m
         5           1.2128            7.87m
         6           1.2052            7.88m
         7           1.1999            7.91m
         8           1.1941            7.87m
         9           1.1897            7.86m
        10           1.1862            7.85m
        20           1.1607            7.67m
        30           1.1498            7.56m
        40           1.1437            7.18m
        50           1.1398            6.84m
        60           1.1373            6.61m
        70           1.1359            6.88m
        80           1.1348            6.91m
        90           1.1338            6.91m
       100           1.1330           

important aspects still to be implemented:
- parameter tuning w/ grid search (train short models on subsets? try param_grid and GridSearch)
- KCV (in combination with above)
- some testing with different preprocessing and feature sets.
- try importing seaborn and doing some EDA on feature importance data?
- test Random Forest version?
- additional feature extraction and tagging (dep relationships and sentence level features)
- If trees are interesting as a final model, XGBoost > sklearn on most problems.